# Classification with pyspark and pytorch lightning

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from plantclef.utils import get_spark

spark = get_spark()
display(spark)

In [ ]:
# Path and dataset names
gcs_path = "gs://dsgt-clef-plantclef-2024/data/process"
dct_emb_train = "training_cropped_resized_v2/dino_dct/data"

# Define the GCS path to the embedding files
dct_gcs_path = f"{gcs_path}/{dct_emb_train}"

# Read the Parquet file into a DataFrame
dct_df = spark.read.parquet(dct_gcs_path)

# Show the data
dct_df.show(n=5, truncate=50)

### convert to a PyTorch dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class EmbeddingDataset(Dataset):
    def __init__(self, data, num_classes):
        self.data = data.toPandas()  # Convert to Pandas DF
        self.num_classes = num_classes  # Total number of classes
        self.species_id = self._get_species_index(data)

    def __len__(self):
        return len(self.data)

    def _get_species_index(self, data):
        species_ids = (
            data.select("species_id").distinct().rdd.map(lambda r: r[0]).collect()
        )
        species_id_to_index = {
            species_id: idx for idx, species_id in enumerate(species_ids)
        }
        return species_id_to_index

    def __getitem__(self, index):
        row = self.data.iloc[index]
        embeddings = torch.tensor(row["dct_embedding"])
        labels = torch.zeros(self.num_classes, dtype=torch.float)
        species_index = self.species_id[row["species_id"]]
        labels[species_index] = 1.0
        return embeddings, labels

In [ ]:
import pytorch_lightning as pl
from torch import nn


class MultiLabelClassifier(pl.LightningModule):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.layer = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return torch.sigmoid(
            self.layer(x)
        )  # Using sigmoid for multi-label classification

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.binary_cross_entropy(
            y_hat, y.float()
        )  # Ensure y is float for BCE
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.002)

### prepare subset of data

In [ ]:
from pyspark.sql import functions as F

# Get a small subset of the dataset
sub_species_df = (
    dct_df.groupBy("species_id")
    .agg(F.count("*").alias("n"))
    .where(F.col("n") > 100)
    .orderBy(F.rand(seed=42))
    .limit(10)
).cache()
sub_species_df.show(truncate=80)

# Collect the species_id into a list of values
species_id_subset = [
    row["species_id"]
    for row in sub_species_df.select("species_id").distinct().collect()
]

# Get subset of images to test pipeline
subset_df = dct_df.where(F.col("species_id").isin(species_id_subset)).cache()
subset_df.show()

# Count number of rows in subset_df
print(f"subset_df count: {subset_df.count()}")

### train the model

In [ ]:
dataset = EmbeddingDataset(data=subset_df, num_classes=10)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# model
model = MultiLabelClassifier(
    num_features=64, num_classes=10
)  # Only using 10 classses for testing

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, dataloader)